In [1]:
import os
import sys
import numpy as np
import time
# import pyaudio
import wave
import zipfile
import wavio
import random
import asyncio
import IPython
from queue import Queue
from threading import Thread
import argparse
import array
np.set_printoptions(threshold=sys.maxsize)

In [2]:
import sounddevice as sd
import soundfile as sf
import scipy

In [3]:
import tensorflow as tf

2024-06-21 10:35:16.593293: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 10:35:17.170845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 10:35:17.170965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 10:35:17.269987: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 10:35:17.469250: I tensorflow/core/platform/cpu_feature_guar

In [4]:
sys.path.append("../ipynb_files/")
sys.path.append("../")
# sys.path.append("../../src/SharedLibs/")

In [5]:
from common import utils as U
from TestSharedLib.bytes import to_bytes, from_bytes, byte_conversion_tests, load_data, load_raw, save_raw, save_scores
from TestSharedLib.constants import quant_support, crops, feature_count
# from data_processing_utility import save_wavefile

In [6]:
from datetime import datetime
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [9]:
# model_path = "../../trained_models/step_1_base_train/tflite_model/uec_model_4Classes_20240520150535.tflite";
model_path = "../../trained_models/step_1_base_train/tflite_model/uec_model_4C_val80tracc89_onesecond_20240621104432.tflite"
tflite_interpreter = tf.lite.Interpreter(model_path=model_path)

In [10]:
# model.summary()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
tflite_interpreter.allocate_tensors();

== Input details ==
name: x.3
shape: [    1     1 20150     1]
type: <class 'numpy.float32'>

== Output details ==
name: 285
shape: [1 4]
type: <class 'numpy.float32'>


### sound preprocessing codes

In [11]:
_inputLen = 20150
_nCrops = 2
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return funcs

def preprocess_debug():
    debug_funcs = []
    debug_funcs += [U.padding( _inputLen// 2),
              # U.normalize(32768.0),]
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return debug_funcs


def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)
    return sound;
    

def padding(pad):
    def f(sound):
        return np.pad(sound, pad, 'constant')

    return f
    

# def random_crop(size):
#     def f(sound):
#         org_size = len(sound)
#         start = random.randint(0, org_size - size)
#         return sound[start: start + size]

#     return f



In [12]:
# _funcs = preprocess_debug()
_funcs = preprocess_setup()

In [15]:
def doSoundClassification(input_wav=None, lblidx=None, channelIdx=0):
    sound = np.float32(wavio.read(input_wav).data.T[0])
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1];
    print(f"get sound signal from {start} to {end}");
    if len(sound)> 220500:
        sound = sound[:220500]
    # if len(sound)> 30225:
    #     sound = sound[:30226]
    # sound = np.int16(preprocess(sound, _funcs));
    sound = preprocess(sound, _funcs)
    print(f"sound info after preprocess:{type(sound)}")
    s_test = np.expand_dims(sound[channelIdx], axis=0)
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    
    tflite_interpreter.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter.invoke()
    pred = tflite_interpreter.get_tensor(output_details[0]['index'])
    print(type(pred))
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"the larget value index is:{pred[0].argmax(axis=0)}")
    print(f"Prediction result: {pred}, and true label idx: {lblidx}")
    # print(f"channel of inpu_wav:{len(sound)}");

In [25]:
test_alarm_wav = "../../datasets/CurrentUse/wav_files/Single_Fold/val/positive/alarm_52/155251_p_3.wav"
test_help_eng_wav = "../Eng_Help_Processed_Test_Sounds/test_man_eng_help_001.wav"
test_help_eng_wav2 = "../../datasets/CurrentUse/wav_files/Single_Fold/val/positive/help_eng_71/s2_extend_to_30225_repeat_help/man_clear_clip_help_032_71_sigrep.wav"
test_

In [26]:
IPython.display.Audio(test_help_eng_wav2)

In [27]:
doSoundClassification(input_wav=test_alarm_wav, lblidx=0, channelIdx=0)

get sound signal from 0 to 99999
sound info after preprocess:<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
the larget value index is:0
Prediction result: [[9.1774160e-01 1.2174783e-04 1.4036530e-05 8.2122691e-02]], and true label idx: 0


In [67]:
# doSoundClassification(input_wav=play_wav1,lblidx=1,channelIdx=1);